In [1]:
from tensorflow import keras

In [2]:
def load_images(directory, data_shape, batch_size=16, validation_split=0.25, seed=2704):
    print(f'Cargando imágenes desde: {directory}')
    image_size = data_shape[0:2]

    train_data = keras.utils.image_dataset_from_directory(
        directory,
        validation_split = validation_split,
        subset = "training",
        # categorical hace que la salida sea sparce (one hot encoding)
        label_mode='categorical',
        seed = seed,
        image_size = image_size,
        batch_size = batch_size,
        shuffle=True)
    
    valid_data = keras.utils.image_dataset_from_directory(
        directory,
        validation_split = validation_split,
        subset = "validation",
        # categorical hace que la salida sea sparce (one hot encoding)
        label_mode='categorical',
        seed = seed,
        image_size = image_size,
        batch_size = batch_size)
    
    print("\n¡Imágenes cargadas exitosamente!")
    print(f"Número de clases encontradas: {len(train_data.class_names)}")
    print(f"Nombres de las clases: {train_data.class_names}")
    
    normalization_model = keras.layers.Rescaling(1./255)
    print("Capa normalizadora creada con éxito") 

    augmentation_model = keras.models.Sequential()
    # aumentamos el número de imágenes haciendo imágenes en espejo vertical y horizontal
    augmentation_model.add(keras.layers.RandomFlip("horizontal_and_vertical")) 
    # aumentamos haciendo rotaciones de las imágenes.
    # factor hace que demos un giro aleatorio desde -0.25 a 0.25 vueltas.
    # fill_mode hace que rellenemos los pixeles vacíos con nearest (rellenamos con el pixel que tenemos mas cerca)
    augmentation_model.add(keras.layers.RandomRotation(factor=0.25, fill_mode='nearest'))
    augmentation_model.add(keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode='nearest'))
    print("Capas de aumento de imágenes creadas con éxito")
    print("No olvides utilizar normalization_model y augmentation_model para mejorar tu base de datos")

    return train_data, valid_data, normalization_model, augmentation_model

In [3]:
def image_augmentation(image, label, augmentation_model):
    image = augmentation_model(image)
    return image, label

In [4]:
def image_normalization(image, label, normalization_model):
    image = normalization_model(image)
    return image, label

In [5]:
def create_resnet_model(input_shape,num_classes):
    backbone = keras.applications.resnet.ResNet50(input_shape=input_shape, weights='imagenet', include_top=False)
    # usaremos un modelo previamente entrenado con sus parámetros aprendidos como funciones para entrenar nuestro nuevo modelo
    for layer in backbone.layers:
        layer.trainable = False
    
    # Construyamos nuestro nuevo modelo capa por capa. Comienza utilizando la capa secuencial; después 
    # agrega el backbone, una capa que aplana nuestra matriz en vectores y, por último, el 
    # encabezado de la clasificación.  No olvides ajustar el número de clases que tu modelo 
    # debe manejar  
   
    model = keras.models.Sequential()
    model.add(backbone) 
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(units=64, activation="relu"))
    model.add(keras.layers.Dense(units=32, activation="relu"))
    model.add(keras.layers.Dense(units=num_classes, activation="softmax"))


    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc']) 
    
    model.summary()
    
    return model
    


In [6]:
def train_model(model, train_data, validation_data, epochs=10):
    model.fit(train_data, validation_data=validation_data, epochs=epochs, verbose=1)
    return model

In [7]:
path = "archive"
data_shape = (150, 150, 3)
batch_size=16
validation_split=0.25
seed=2704

# Cargamos los datos
train_data, valid_data, normalization_model, augmentation_model = load_images(directory=path, data_shape=data_shape, batch_size=batch_size, validation_split=validation_split, seed=seed)
num_classes = len(train_data.class_names)

# Tratamiento a datos de entrenamiento
train_data = train_data.map(lambda image, label: image_normalization(image, label, normalization_model))
train_data = train_data.map(lambda image, label: image_augmentation(image, label, augmentation_model))

# Tratamiento a datos de validación
valid_data = valid_data.map(lambda image, label: image_normalization(image, label, normalization_model))

# Creamos modelo ResNet
model = create_resnet_model(input_shape=data_shape, num_classes=num_classes)

# Entrenamos el modelo
model = train_model(model=model, train_data=train_data, validation_data=valid_data, epochs=10)

Cargando imágenes desde: archive
Found 70549 files belonging to 15 classes.
Using 52912 files for training.
Found 70549 files belonging to 15 classes.
Using 17637 files for validation.

¡Imágenes cargadas exitosamente!
Número de clases encontradas: 15
Nombres de las clases: ['Apple', 'Banana', 'Carambola', 'Guava', 'Kiwi', 'Mango', 'Orange', 'Peach', 'Pear', 'Persimmon', 'Pitaya', 'Plum', 'Pomegranate', 'Tomatoes', 'muskmelon']
Capa normalizadora creada con éxito
Capas de aumento de imágenes creadas con éxito
No olvides utilizar normalization_model y augmentation_model para mejorar tu base de datos


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 5, 5, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,721,423 (90.49 MB)

 Trainable params: 133,711 (522.31 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 2234s 671ms/step - acc: 0.2708 - loss: 2.3874 - val_acc: 0.3272 - val_loss: 2.2290
Epoch 2/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 1904s 576ms/step - acc: 0.3251 - loss: 2.2431 - val_acc: 0.3353 - val_loss: 2.1914
Epoch 3/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 1925s 582ms/step - acc: 0.3272 - loss: 2.2148 - val_acc: 0.3374 - val_loss: 2.1771
Epoch 4/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 2106s 637ms/step - acc: 0.3301 - loss: 2.1960 - val_acc: 0.3410 - val_loss: 2.1753
Epoch 5/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 2715s 821ms/step - acc: 0.3307 - loss: 2.1839 - val_acc: 0.3426 - val_loss: 2.1570
Epoch 6/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 2433s 735ms/step - acc: 0.3343 - loss: 2.1704 - val_acc: 0.3435 - val_loss: 2.1366
Epoch 7/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 3334s 1s/step - acc: 0.3362 - loss: 2.1623 - val_acc: 0.3550 - val_loss: 2.1526
Epoch 8/10
3307/3307 ━━━━━━━━━━━━━━━━━━━━ 1901s 575ms/step - acc: 0.3373 - loss: 2.1526 - val_acc: 0.3531 - val_loss: 2.1730
Epo